<a href="https://colab.research.google.com/github/pvoipt/AiDeveloping/blob/main/TickNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

학번 : 30503      이름 : 고도현 <br>
작품 : 뇌 구조를 모방한 신경망 <br>
버전 : 0.5 / 20210616 <br>
<p>
개선할 부분 <br>
- 유전 알고리즘으로 학습하기에는 너무 시간이 오래 걸리고 효과도 미미함. 효과를 볼 정도로 개체수를 많게 하면 하루종일 연산해야 할 듯함. <br>
- 병렬 처리를 일부분에만 적용함. 전체적으로 적용해야 함. <br>
- 주석이 부족함. <br>
- 뉴런과 퍼셉트론이라는 용어를 혼용했음. 통일해야 함. <br>
- 뉴런(노드) 수를 줄이기. <br>
</p>
<br>
<p>
추가사항 <br>
- 손글씨 인식을 CNN등의 방법을 적용해보기. <br>
- 연결의 가중치를 바꾸는 학습만 시도하고 연결을 새로 만들거나 없애는 부분이 없음. <br>

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical  # one-hot encoding 을 위한 함수
from tensorflow.keras.models import Sequential  # 레이어를 층층히 쌓아가는 연쇄 모델
from tensorflow.keras.layers import Dense  # 완전연결층
from tensorflow.keras.models import load_model  # 저장된 모델 불러오기

In [ ]:
import multiprocessing as mp
mp.cpu_count()

To Do List
 - 병렬 처리(가장 시간이 오래 걸리는 출력값 계산 부분에는 적용함)
 - 학습 방법 개선 (여러개의 토큰을 입력해서 출력층에 도달한 토큰이 지나온 길을 강화 등)

In [ ]:
def toGray(n):
	return bin(n ^ (n >> 1))
 
def toBin(n):
	bi = n ^ (n >> 1)
	bi = bi ^ (bi >> 2)
	bi = bi ^ (bi >> 4)
	bi = bi ^ (bi >> 8)
	bi = bi ^ (bi >> 16)
	return bin(bi)

In [ ]:
def stepF(X):
  if np.sum(X) > 0.3:
    return 1
  else:
    return 0

In [ ]:
def ReLU(X):
  return np.maximum(0, np.sum(X))

In [ ]:
def SSE(y, t):    #sum of squares for error, 오차제곱합
  return 0.5 * np.sum((y - t)**2)

In [ ]:
def randGene(n):      #무작위 유전자 생성
  temp = np.random.binomial(1, 0.5, n)
  r = ''
  for i in temp:
    r += str(i)
  return r

In [ ]:
def GTV(gene):  #Gene To Value


  if isinstance(gene, np.float64):
    return 1
  else:
    bi = ''
    for i in gene:
      bi += str(i)
    return int(bi, 2)

In [ ]:
class Perceptron:
  def __init__(self, size, nSpN, maxInput, role, sumType, nextSize):
    self.role = role
    #if role == "sensory":       #감각뉴런이라면 가중치 1 (원본 그대로 반영)
      #self.wgtGene = np.ones(100)
    if role == "inter":       #연결뉴런이라면 가중치 0~1 랜덤
      self.wgtGene = []
      for i in range(maxInput):
        self.wgtGene.append(randGene(8)) #가중치 유전자 생성 (그레이코드)
    elif role == "motor":
      pass
    self.size = size
    if nextSize == -1:
      self.nextSize = size
    else:
      self.nextSize = nextSize
    self.sumType = sumType
    self.nSpN = nSpN    # nSpN = the number of Synapse per Neuron : 뉴런별로 뻗어나갈 시냅스 수
    self.X = []       #입력값 받는 리스트
    self.summation = 0      #가중치을 더하는 변수 (시간적 가중 용)
    self.maxInput = maxInput #최대 입력(최대 연결될 수 있는 시냅스 수)(넘어가면 무시)
  def connectNeuron(self):
    self.cN = np.random.randint(0, self.nextSize, self.nSpN)    #랜덤하게 시냅스 연결
  def calWgt(self):       #가중치 유전자를 값으로 변환
    self.wgt  = []
    for i in range(self.maxInput):
      self.wgt = np.append(self.wgt, int(toBin(int(self.wgtGene[i], 2)), 2)/ 2**8)
  def input_(self, X):    #값 입력받음
    if len(self.X) + len(X) < self.maxInput or self.role == "motor":
      self.X += X
  def out(self):
    if self.role == "sensory":
      self.WX = self.X
    if self.role == "inter":
      self.WX = np.array(self.X) * self.wgt[:len(self.X)]   #가중치와 입력 곱함
      self.X = []
    if self.role == "motor":
      return 0
    else:
      if self.sumType == "spatial":      #공간적/시간적 가중에 따라 활성화 방식 다름
        return stepF(self.WX)
      elif self.sumType == "temporal":
        self.summation += np.sum(self.WX)
        if self.summation >= 1:
          self.summation -= 1
          return 1
        else:
          return 0

In [ ]:
class Network:
  def __init__(self, size, nSpN, maxInput, nInputLayer, nOutputLayer): #nSpN = the number of Synapse per Neuron
    self.size = size
    self.nSpN = nSpN
    self.maxInput = maxInput
    self.inputLayer = []
    self.neuron = []
    self.nInputLayer = nInputLayer
    self.nOutputLayer = nOutputLayer
    self.inputTable = []
  def makeup(self):
    for i in range(self.nInputLayer):         #입력뉴런(감각뉴런) 생성
      n = Perceptron(self.size, self.nSpN, self.maxInput, "sensory", "temporal", self.size)
      n.connectNeuron()
      self.inputLayer.append(n)
    for i in range(self.size):      #연결뉴런 생성
      n = Perceptron(self.size, self.nSpN, self.maxInput, "inter", "temporal", self.size + self.nOutputLayer)
      n.connectNeuron()
      self.neuron.append(n)
    for i in range(self.nOutputLayer):
      n = Perceptron(self.size, self.nSpN, self.maxInput, "motor", "temporal", -1)
      self.neuron.append(n)
  def calNeuronWgt(self):
    for i in range(self.size):
      self.neuron[i].calWgt()
  def input_(self):     #입력층 입력
    for i in range(self.size + self.nOutputLayer):
      self.inputTable.append([])
    for i in range(self.nInputLayer):
      r = self.inputLayer[i].out()
      for j in range(self.nSpN):
        self.inputTable[self.inputLayer[i].cN[j]].append(r)
  def multiTick(self, start, end):      #병렬 처리를 위해 tick()에서 분리한 함수
    for i in range(start, end):
      r = self.neuron[i].out()
      for j in range(self.nSpN):
        self.inputTable[self.neuron[i].cN[j]].append(r)    
  def tick(self):       #시간 경과 (틱)
    procs = []
    start = 0
    part = int(self.size / mp.cpu_count())        #병렬 처리 지정
    end = part
    for i in range(mp.cpu_count()):
      proc = multiprocessing.Process(target = self.multiTick, args = (start, end))
      procs.append(proc)
      start = end + 1
      end += part
      proc.start()
    for proc in procs:
      proc.join()

    for i in range(self.size + self.nOutputLayer):
      if len(self.inputTable[i]):
        self.neuron[i].input_(self.inputTable[i])
    self.inputTable = []
    for i in range(self.size + self.nOutputLayer):
      self.inputTable.append([])
  def output(self):     #출력층 출력
      self.out = []
      for i in range(self.size, self.size + self.nOutputLayer):
        self.out.append(np.sum(self.neuron[i].X))
      return np.array(self.out) / np.sum(self.out)

In [ ]:
def breed(p1, p2):        #자손 생성 함수 p1 : 부모 1, p2 : 부모 2
  offspring = p1
  #newGene = []

  for i in range(p1.size):
    newChromosome = []
    for j in range(p1.maxInput):
      newGene = ''
      for k in range(8):
        rand = np.random.binomial(1, 0.5, 1)[0]    #50% 확률 구현
        if rand:
          newGene += (p1.neuron[i].wgtGene[j][k])
        else:
          newGene += (p2.neuron[i].wgtGene[j][k])
      newChromosome.append(newGene)

  for i in range(offspring.size):
    offspring.neuron[i].wgtGene = newChromosome
  
  return offspring
  

In [ ]:
#MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

input_shape = X_train.shape[1] * X_train.shape[2] #그림 크기 (28*28)
number_of_classes = len(set(y_train))

X_train = X_train / 255.0  #정규화
X_test = X_test / 255.0
X_train = X_train.reshape(-1, input_shape) #차원 낮추기
X_test = X_test.reshape(-1, input_shape)

y_train = to_categorical(y_train, number_of_classes) #원-핫 인코딩
y_test = to_categorical(y_test, number_of_classes)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
print(np.shape(X_train))

X_train[0][550]

(60000, 784)


0.9803921568627451

In [ ]:
def test(n):
  total = []
  for a in range(len(n)):
    t = 0
    for i in range(10):
      for j in range(784):
        n[a].inputLayer[j].input_([X_train[i][j]])
      for k in range(10):
        n[a].input_()
        n[a].tick()
      t += SSE(y_train[i], n[a].output())
    total.append(t)
  return total

In [ ]:
def shift_of_generation(n):
  for i in range(len(n)):
    n[i].calNeuronWgt()
  total = test(n)
  print(total)
  p1 = np.argmin(total)
  del total[p1]
  p2 = np.argmin(total)
  o = []
  for i in range(10):             #자손 10 생성
    o.append(breed(n[p1], n[p2]))   
  return o  


In [ ]:
n = []
for i in range(10):
  n.append(Network(10000, 10, 30, 784, 10))
  n[i].makeup()

gen5 = shift_of_generation(shift_of_generation(shift_of_generation(shift_of_generation(shift_of_generation(shift_of_generation(n))))))